## set directory & import

In [1]:
from google.colab import drive

In [2]:
# Mount your drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [3]:
# Change directory
%cd /content/drive/MyDrive/MenuAnalysis - Pilot (gdrive)/nature_food/USA/State&County

/content/drive/.shortcut-targets-by-id/14151HLaDbRvLarI0syLDYkmR8C1KkHbW/MenuAnalysis - Pilot (gdrive)/nature_food/USA/State&County


In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import random
import scipy.stats
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

In [5]:
from scipy.spatial.distance import cosine
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from tabulate import tabulate
from sklearn.metrics import r2_score

In [6]:
#from sklearn.model_selection import GridSearchCV, GroupKFold
#from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import RocCurveDisplay, accuracy_score, precision_score, f1_score, recall_score
from sklearn.metrics import roc_curve, auc, confusion_matrix, roc_auc_score, mean_squared_error, mean_absolute_error
from scipy.stats import spearmanr
from sklearn.model_selection import KFold
import scipy.stats as st
from scipy import stats
from statistics import stdev

## Visualization

### Creat FEND from RND

In [7]:
# 2020v complete
county_ho=pd.read_csv('PLACES__County_Data__GIS_Friendly_Format___2020_release.csv')

In [8]:
# Change dtype
county_ho=county_ho.astype({'CountyFIPS': 'str'})
# Make it 5-digit number
for i in tqdm(range(len(county_ho))):
    if len(county_ho['CountyFIPS'][i]) != 5:
        county_ho['CountyFIPS'][i] = '0' + county_ho['CountyFIPS'][i]

100%|██████████| 3142/3142 [00:00<00:00, 6667.42it/s]


In [9]:
county_boundary = gpd.read_file('./cb_2015_us_county_500k/cb_2015_us_county_500k.shp')
county_boundary['CountyFIPS']=county_boundary['STATEFP']+county_boundary['COUNTYFP']
county_boundary = county_boundary.rename(columns={"NAME":"CountyName", "STUSPS": "StateAbbr"})
county_boundary= county_boundary.to_crs({'init': 'EPSG:4326'})

/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [10]:
# merge
county_ho = county_ho.merge(county_boundary,how='left', on=['CountyFIPS'])
# To gpd
county_ho = gpd.GeoDataFrame(county_ho, crs="EPSG:4326", geometry='geometry')

In [11]:
county_ho[county_ho['geometry'].isna()]

,StateAbbr,StateDesc,CountyName_x,CountyFIPS,TotalPopulation,ACCESS2_CrudePrev,ACCESS2_Crude95CI,ACCESS2_AdjPrev,ACCESS2_Adj95CI,ARTHRITIS_CrudePrev,...,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,CountyName_y,LSAD,ALAND,AWATER,geometry


In [12]:
# Read RND data
ALL_RND = pd.read_csv('/content/drive/MyDrive/MenuAnalysis - Pilot (gdrive)/nature_food/data/ALL_RND_pp98pct.csv')
# APP_RND = pd.read_csv('/content/drive/MyDrive/MenuAnalysis - Pilot (gdrive)/nature_food/data/APP_RND_pp98pct.csv')
# MAIN_RND = pd.read_csv('/content/drive/MyDrive/MenuAnalysis - Pilot (gdrive)/nature_food/data/MAIN_RND_pp98pct.csv')
# DSRT_RND = pd.read_csv('/content/drive/MyDrive/MenuAnalysis - Pilot (gdrive)/nature_food/data/DSRT_RND_pp98pct.csv')
# DRNK_RND = pd.read_csv('/content/drive/MyDrive/MenuAnalysis - Pilot (gdrive)/nature_food/data/DRNK_RND_pp98pct.csv')

In [13]:
# ALL
valid_points = gpd.GeoDataFrame(ALL_RND, geometry=gpd.points_from_xy(ALL_RND.location_longitude, ALL_RND.location_latitude))
#ct.crs
valid_points.crs = 'EPSG:4326'
ALL_RND_ct = gpd.sjoin(valid_points, county_ho, op = 'within')

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [14]:
# ALL MENU
ALL_FEND_ct=ALL_RND_ct.groupby(['CountyFIPS']).median(numeric_only=True)[['ALL_RND_pred','lower_ci','upper_ci',
                                                         'location_latitude','location_longitude']]
ALL_FEND_ct['CountyFIPS']=ALL_FEND_ct.index
ALL_FEND_ct = ALL_FEND_ct.reset_index(drop=True)
ALL_FEND_ct= ALL_FEND_ct.rename(columns={'ALL_RND_pred':'ALL_FEND_pred'})
# count
ALL_FEND_n=ALL_RND_ct.groupby(['CountyFIPS']).count()[['ALL_RND_pred']]
ALL_FEND_n['CountyFIPS']=ALL_FEND_n.index
ALL_FEND_n = ALL_FEND_n.reset_index(drop=True)
ALL_FEND_n = ALL_FEND_n.rename(columns={'ALL_RND_pred':'restaurant_count'})
ALL_FEND_ct=ALL_FEND_ct.merge(ALL_FEND_n[['CountyFIPS','restaurant_count']], how='left', on=['CountyFIPS'])
# max
ALL_FEND_max=ALL_RND_ct.groupby(['CountyFIPS']).max(numeric_only=True)[['ALL_RND_pred']]
ALL_FEND_max['CountyFIPS']=ALL_FEND_max.index
ALL_FEND_max = ALL_FEND_max.reset_index(drop=True)
ALL_FEND_max = ALL_FEND_max.rename(columns={'ALL_RND_pred':'ALL_FEND_max'})
ALL_FEND_ct=ALL_FEND_ct.merge(ALL_FEND_max[['CountyFIPS','ALL_FEND_max']], how='left', on=['CountyFIPS'])
# min
ALL_FEND_min=ALL_RND_ct.groupby(['CountyFIPS']).min(numeric_only=True)[['ALL_RND_pred']]
ALL_FEND_min['CountyFIPS']=ALL_FEND_min.index
ALL_FEND_min = ALL_FEND_min.reset_index(drop=True)
ALL_FEND_min = ALL_FEND_min.rename(columns={'ALL_RND_pred':'ALL_FEND_min'})
ALL_FEND_ct=ALL_FEND_ct.merge(ALL_FEND_min[['CountyFIPS','ALL_FEND_min']], how='left', on=['CountyFIPS'])
# std
ALL_FEND_std=ALL_RND_ct.groupby(['CountyFIPS']).std(numeric_only=True)[['ALL_RND_pred']]
ALL_FEND_std['CountyFIPS']=ALL_FEND_std.index
ALL_FEND_std = ALL_FEND_std.reset_index(drop=True)
ALL_FEND_std = ALL_FEND_std.rename(columns={'ALL_RND_pred':'ALL_FEND_std'})
ALL_FEND_std['ALL_FEND_std'].fillna(0, inplace = True)
ALL_FEND_ct=ALL_FEND_ct.merge(ALL_FEND_std[['CountyFIPS','ALL_FEND_std']], how='left', on=['CountyFIPS'])
# merge with gpd
ALL_FEND_ct=ALL_FEND_ct.merge(county_ho[['CountyFIPS','StateAbbr','CountyName_x','TotalPopulation','OBESITY_AdjPrev','DIABETES_AdjPrev',
                                         'HIGHCHOL_AdjPrev','CHD_AdjPrev','BPHIGH_AdjPrev','geometry']], how='left', on=['CountyFIPS'])

In [15]:
# 98pct #ALL
print('Correlation between FEND-county and Obesity:\n>',scipy.stats.spearmanr(ALL_FEND_ct['ALL_FEND_pred'],ALL_FEND_ct['OBESITY_AdjPrev']))
print('\nCorrelation between FEND-county and Diabetes:\n>',scipy.stats.spearmanr(ALL_FEND_ct['ALL_FEND_pred'],ALL_FEND_ct['DIABETES_AdjPrev']))
print('\nCorrelation between FEND-county and High Cholesterol:\n>',scipy.stats.spearmanr(ALL_FEND_ct['ALL_FEND_pred'],ALL_FEND_ct['HIGHCHOL_AdjPrev']))
print('\nCorrelation between FEND-county and High Blood Pressure:\n>',scipy.stats.spearmanr(ALL_FEND_ct['ALL_FEND_pred'],ALL_FEND_ct['BPHIGH_AdjPrev']))
print('\nCorrelation between FEND-county and CHD(Heart Disease):\n>',scipy.stats.spearmanr(ALL_FEND_ct['ALL_FEND_pred'],ALL_FEND_ct['CHD_AdjPrev']))

Correlation between FEND-county and Obesity:
> SignificanceResult(statistic=-0.252533232605858, pvalue=8.672767438638669e-38)

Correlation between FEND-county and Diabetes:
> SignificanceResult(statistic=-0.23280146041575475, pvalue=3.243296394192709e-32)

Correlation between FEND-county and High Cholesterol:
> SignificanceResult(statistic=-0.23153162020549106, pvalue=7.119789178455298e-32)

Correlation between FEND-county and High Blood Pressure:
> SignificanceResult(statistic=-0.2667930459419295, pvalue=3.918365348529497e-42)

Correlation between FEND-county and CHD(Heart Disease):
> SignificanceResult(statistic=-0.2732978606623796, pvalue=3.3156607017705283e-44)


### Or read dataset

In [16]:
ALL_FEND_ct = pd.read_csv('ALL_FEND_county_level_98pct_all.csv')
# To geo df
ALL_FEND_ct['geometry'] = ALL_FEND_ct['geometry'].apply(wkt.loads)
ALL_FEND_ct = gpd.GeoDataFrame(ALL_FEND_ct, crs="EPSG:4326", geometry='geometry')

In [18]:
ALL_FEND_ct

,ALL_FEND_pred,lower_ci,upper_ci,location_latitude,location_longitude,CountyFIPS,restaurant_count,ALL_FEND_max,ALL_FEND_min,ALL_FEND_std,...,median_income,pp_publicTP,pp_longcomute,pp_lowskillJob_F,pp_lowskillJob_M,pp_collegeEd_F,pp_collegeEd_M,employment_rate,pp_lowskillJob,pp_collegeEd
0,0.691344,0.681026,0.704249,32.46030,-86.42790,1001,45,1.002311,0.289552,0.129658,...,58731.0,0.006576,0.111111,0.231903,0.528346,0.341139,0.359389,0.963234,0.528346,0.350264
1,0.685290,0.673165,0.695381,30.39855,-87.68940,1003,436,1.265305,0.227900,0.147123,...,58320.0,0.000483,0.172500,0.263823,0.532513,0.424609,0.400504,0.957449,0.532513,0.412557
2,0.724260,0.715622,0.732331,31.88305,-85.14870,1005,4,0.951188,0.630371,0.139237,...,32525.0,0.003280,0.142082,0.383718,0.697092,0.226851,0.153524,0.908335,0.697092,0.190188
3,0.664255,0.655289,0.673960,33.95150,-86.46905,1009,4,0.839973,0.646458,0.091312,...,49358.0,0.001291,0.390525,0.295672,0.680773,0.286724,0.223785,0.966358,0.680773,0.255254
4,0.668088,0.652593,0.676030,31.84800,-86.63880,1013,3,0.907564,0.636619,0.148184,...,40688.0,0.001535,0.161574,0.364610,0.641265,0.283626,0.173156,0.931954,0.641265,0.228391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2503,0.694360,0.685795,0.703624,41.57900,-109.24600,56037,29,0.946113,0.539763,0.100638,...,74843.0,0.028884,0.100401,0.240179,0.685370,0.381875,0.311695,0.939076,0.685370,0.346785
2504,0.717765,0.702955,0.734198,43.48070,-110.77500,56039,43,0.972125,0.283677,0.137739,...,84678.0,0.034974,0.026608,0.174924,0.395582,0.646555,0.584865,0.989237,0.395582,0.615710
2505,0.662168,0.651761,0.671840,41.25780,-110.97000,56041,3,0.670830,0.632750,0.019961,...,63403.0,0.032987,0.180469,0.243120,0.664354,0.251082,0.277880,0.937179,0.664354,0.264481
2506,0.676914,0.668621,0.685881,44.02015,-107.95350,56043,2,0.679539,0.674288,0.003713,...,54158.0,0.000000,0.030182,0.257299,0.644163,0.346043,0.323572,0.960824,0.644163,0.334808


In [19]:
ALL_FEND_ct['ALL_FEND_pred'].describe()

count    2508.000000
mean        0.704569
std         0.051158
min         0.355560
25%         0.681504
50%         0.698738
75%         0.719073
max         0.978693
Name: ALL_FEND_pred, dtype: float64

In [20]:
mapbox_access_token = "pk.eyJ1IjoibWFwYm94c2VvIiwiYSI6ImNsZHo0ejU0dzBxMHAzb292Ym41Yzk4bzMifQ.ZYVGCdm8E2kH2QGi2gd9ng"

fig = px.choropleth_mapbox(ALL_FEND_ct, geojson=ALL_FEND_ct.geometry, locations=ALL_FEND_ct.index, color='ALL_FEND_pred',
                           color_continuous_scale="RdBu",
                           range_color=(ALL_FEND_ct['ALL_FEND_pred'].quantile(0.25),ALL_FEND_ct['ALL_FEND_pred'].quantile(0.75)),
                           mapbox_style="dark",
                           opacity=1,
                           #labels={'Predicted RND':'RND'}
                          )
fig.update_layout(margin={"r":0,"l":0,"b":0})
fig.update_traces(marker_line_width=0)
fig.update_layout(
    autosize=True,
    hovermode='closest',
    title_text ='FEND-ALL for Restaurants in US census tracts',
    title_x =0.5,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=40.284945,
            lon= -97.673587
        ),
        pitch=0,
        zoom=3
    ),
)

fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [26]:
fig = px.choropleth_mapbox(ALL_FEND_ct, geojson=ALL_FEND_ct.geometry, locations=ALL_FEND_ct.index, color=np.log10(ALL_FEND_ct['restaurant_count']),
                           color_continuous_scale="thermal",
                           range_color=(0,3),
                           mapbox_style="dark",
                           opacity=1,
                           labels={'color':'log_restaurant_count'}
                          )
fig.update_layout(margin={"r":0,"l":0,"b":0})
fig.update_traces(marker_line_width=0)
fig.update_layout(
    autosize=True,
    hovermode='closest',
    title_text ='Number of Restaurants in US census tracts',
    title_x =0.5,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=40.284945,
            lon= -97.673587
        ),
        pitch=0,
        zoom=3
    ),
)

fig.show()

Output hidden; open in https://colab.research.google.com to view.